# Gemma

In [ ]:
# This is an example notebook showing how to compute perplexity using Gemma. 
# This notebook has to be adopted based on what model has been used for story generation. It is suggested to use the same model for story generation and perplexity calculation. 

In [1]:
import os
from transformers import pipeline
import transformers
import torch
os.environ['HF_TOKEN']="Enter your Hugging Face access token/API key"
os.environ['HUGGINGFACEHUB_API_TOKEN']="Enter your Hugging Face access token/API key"

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [3]:
#Example text to check if the model has been loaded properly. 
input_text = "What is Machine Learning?"
input_ids = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<bos>What is Machine Learning?

Machine learning is a branch of artificial intelligence that focuses on the development


In [4]:
# Ensure the model is in evaluation mode
model.eval()

# Move model to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaR

In [5]:
def calculate_perplexity(text):
    # Encode the input text
    inputs = tokenizer(text, return_tensors="pt").to(device)

    # Generate output using labels for loss calculation
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        log_likelihood = outputs.loss.item()

    # Calculate perplexity normalized by the number of tokens
    perplexity = torch.exp(torch.tensor(log_likelihood / inputs["input_ids"].shape[1])).item()
    return perplexity

# Example text to evaluate
input_text = "Hi how are you"

# Calculate and print perplexity
perplexity = calculate_perplexity(input_text)
print(f"Perplexity: {perplexity}")

Perplexity: 2.2426650524139404


In [ ]:
# Code to compute perplexity for all generated stories.
# Assuming all stories are loaded from a CSV file which has the column "FinalGeneratedStories"
import pandas as pd
path = "Enter path to your CSV file."
df = pd.read_csv(path)
perplexity_values = []
for i in range (len(df)):
    story = df.iloc[i]["FinalGeneratedStory"]
    perplexity = calculate_perplexity(story)
    perplexity_values.append(perplexity)

#Save all preplexity values back to the dataframe and then to the CSV. 
df['Gemma_perplexity'] = perplexity_values
df.to_csv(path, index=False)